In [1]:
# DL needs
import tensorflow as tf
import keras as kr

# Data needs
import pandas as pd
from sklearn.model_selection import train_test_split

# Numerical computation needs
import numpy as np

# plotting needs
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# ensuring reproducibility
random_seed=42
tf.random.set_seed(random_seed)

# for os interactions
import os

2025-03-11 19:11:09.165422: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# setup paths to data directory
train_dir='10_food_classes_all_data/train'
test_dir='10_food_classes_all_data/test'

# getting the sub-dirs to get classnames
import pathlib
datadir = pathlib.Path(train_dir)
class_names=np.array(sorted([item.name for item in datadir.glob("*")]))
print(class_names)

['chicken_curry' 'chicken_wings' 'fried_rice' 'grilled_salmon' 'hamburger'
 'ice_cream' 'pizza' 'ramen' 'steak' 'sushi']


In [8]:
# Pre-process the data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# initializing ImageDataGenerator with augmentation
train_datagen_augmented=ImageDataGenerator(rescale = 1/255.0)
valid_datagen=ImageDataGenerator(rescale=1/255.0,)

train_data=train_datagen_augmented.flow_from_directory(directory=train_dir,
                                             batch_size=32, # mini batch size
                                             target_size=(224,224), # target size of images (h,w) 
                                             class_mode="categorical", # 'categorical' (default), 'binary',..
                                             seed = random_seed,
                                             shuffle=True 
                                             )

valid_data=valid_datagen.flow_from_directory(directory=test_dir,
                                             batch_size=32,
                                             target_size=(224,224),
                                             class_mode="categorical",
                                             seed = random_seed,
                                             shuffle=True 
                                             )


Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [16]:
# creating a CNN model (baseline: tinyVGG)

@kr.saving.register_keras_serializable(package="tinyVGG")
class tinyVGG(kr.Model):
    def __init__(self,lr=0.001,metrics=['accuracy'],**kwargs):
        # calling constructor of kr.Model() class
        super().__init__(**kwargs)

        # defining model layers

        ### CONV-POOL 1
        self.cnn1 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,padding='valid',
                                           input_shape=(224,224,3),name='conv_1')
        self.cnn2 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,name='conv_2')
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=2,
                                               padding='valid',name='pool_1')
        
        ### CONV-POOL 2
        self.cnn3 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,name='conv_3')
        self.cnn4 = tf.keras.layers.Conv2D(filters=10,kernel_size=3,name='conv_4')
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=2,name='pool_2')

        ### FLATTEN
        self.flatten1 = tf.keras.layers.Flatten(name='flatten_1')

        ### output layer
        self.output_layer = tf.keras.layers.Dense(10,name='output_layer')


        # layers:
        self._layers = [self.cnn1, self.cnn2, self.pool1,
                       self.cnn3, self.cnn4, self.pool2,
                       self.flatten1,
                       self.output_layer
                       ]
        
        # activations
        self.activations = {
            'conv_1':'relu',
            'conv_2':'relu',
            'pool_1':None,
            'conv_3':'relu',
            'conv_4':'relu',
            'pool_2':None,
            'flatten_1':None,
            'output_layer':'softmax'
        }

        # outputs & tracking
        self.track_layer_output = False
        self.track_activation_output = False
        self.layer_outputs=[]
        self.activation_outputs=[]
        

        ### other hyperparams
        self.loss_fn = 'categorical_crossentropy',
        self.optimizer = kr.optimizers.Adam(learning_rate=lr)
        self.loss_metrics = metrics
        

        ### build the model
        self.build()

    
    def build(self):
        inputs=tf.keras.Input(shape=[224,224,3])
        x=self._layers[0](inputs)
        for layer in self._layers[1:]:
            x=layer(x)

    def compile(self):
        super().compile(optimizer=self.optimizer,loss=self.loss_fn,metrics=self.loss_metrics)
    
    def call(self,inputs):
        x = inputs
        layer_outputs=[]
        activation_outputs=[]

        for layer in self._layers:
            # forward pass and appending output of each layer and activation
            x = layer(x)
            
            if self.track_layer_output:
                layer_outputs.append(x)

            x = self.get_activation(layer_name=layer.name)(x)
            
            if self.track_activation_output:
                activation_outputs.append(x)

        self.layer_outputs=layer_outputs
        self.activation_outputs=activation_outputs

        output = x
        return output


    def get_activation(self,layer_name):
        activation_name = self.activations.get(layer_name,None)
        def activation(input):
            if activation_name == None:
                    return input
            else:
                function = getattr(tf.nn,activation_name,None)
                return input if function is None else function(input)
        return activation


In [19]:
model_1=tinyVGG(name='model_1')
model_1.summary()

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 222, 222, 10)   │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 220, 220, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 110, 110, 10)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 108, 108, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_4 (Conv2D)                 │ (None, 106, 106, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 53, 53, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 28090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │       280,910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 283,920 (1.08 MB)

 Trainable params: 283,920 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# fitting the model
model_1.compile()
history_1 = model_1.fit(train_data,
            epochs=1,
            steps_per_epoch=len(train_data),
            validation_data=valid_data,
            validation_steps=len(valid_data)
            )

235/235 ━━━━━━━━━━━━━━━━━━━━ 164s 695ms/step - accuracy: 0.1950 - loss: 2.1833 - val_accuracy: 0.3224 - val_loss: 1.9159


In [22]:
model_1.save("best_model.keras")

In [24]:
model_1.evaluate(valid_data)

79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 97ms/step - accuracy: 0.3342 - loss: 1.8892


[1.9158892631530762, 0.3224000036716461]

In [23]:
m=kr.models.load_model('best_model.keras')
m.summary()

/home/rudraksha14/.local/lib/python3.10/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/home/rudraksha14/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_1 (Conv2D)                 │ (None, 222, 222, 10)   │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2 (Conv2D)                 │ (None, 220, 220, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 110, 110, 10)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3 (Conv2D)                 │ (None, 108, 108, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_4 (Conv2D)                 │ (None, 106, 106, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 53, 53, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 28090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 10)             │       280,910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 283,920 (1.08 MB)

 Trainable params: 283,920 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
m.compile()

In [28]:
m.evaluate(valid_data)

79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - accuracy: 0.3158 - loss: 1.9488


[1.9158889055252075, 0.3224000036716461]